<a href="https://colab.research.google.com/github/daniaesterfs/Chatbot/blob/master/chatbot_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch

Main.py

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np

stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stemming(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenize_sentence, all_words):
    tokenize_sentence = [stemming(w) for w in tokenize_sentence]

    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenize_sentence:
            bag[idx] = 1.0

    return bag

# sentence = ["Hello", "How", "are", "you"] 
# words = ["hi", "hello", "I", "see", "you"]
# bag = bag_of_words(sentence, words)
# print(bag)



# words = ["Organizing", "organizes", "organize"]
# stemmed_words = [stemming(w) for w in words]
# print(stemmed_words)



Model.py

In [ ]:
import torch
import torch.nn as nn

class neuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(neuralNetwork, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)

        return out

TRAIN.PY

In [ ]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# from nltk_utils import bag_of_words, tokenize, stem
# from model import NeuralNet

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stemming(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = neuralNetwork(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')


data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags" : tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'Training complete with data saved in {FILE}')

15 patterns
4 tags: ['Pay', 'goodbye', 'greeting', 'transfer']
33 unique stemmed words: ["'s", ',', 'anyon', 'are', 'day', 'do', 'find', 'good', 'hard', 'hello', 'hi', 'how', 'i', 'is', 'it', 'know', 'later', 'money', 'my', "n't", 'okay', 'pay', 'payment', 'see', 'some', 'thankyou', 'there', 'to', 'transfer', 'up', 'well', 'what', 'you']
33 4
Epoch [100/1000], Loss: 0.7855
Epoch [200/1000], Loss: 0.2345
Epoch [300/1000], Loss: 0.0646
Epoch [400/1000], Loss: 0.0228
Epoch [500/1000], Loss: 0.0113
Epoch [600/1000], Loss: 0.0046
Epoch [700/1000], Loss: 0.0030
Epoch [800/1000], Loss: 0.0029
Epoch [900/1000], Loss: 0.0024
Epoch [1000/1000], Loss: 0.0011
final loss: 0.0011
Training complete with data saved in data.pth


CHAT.PY

In [ ]:
import random
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as f:
  intents = json.load(f)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

model = neuralNetwork(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Sinarmas"
print("Let's chat! type 'quit' to exit")
while True:
  sentence = input('You: ')
  if sentence == "quit":
    break

  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X)

  output = model(X)
  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs  = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]

  if prob.item() > 0.75:
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        print(f"{bot_name}: {random.choice(intent['responses'])}")
  else:
    print(f"{bot_name}: I do not understand...")



Let's chat! type 'quit' to exit
You: HELLO
Sinarmas: Good to see you again!
You: hi
Sinarmas: Hi, there, How can I help?
You: how to transfer money
Sinarmas: You can transfer money by pressing the Send button
You: thanks
Sinarmas: I do not understand...
You: thankyou
Sinarmas: Good to see you again!
You: bye
Sinarmas: I do not understand...


KeyboardInterrupt: ignored

USING ATTENTION MODEL

In [ ]:
import keras
from keras import activations, initializers, regularizers, constraints
from keras.engine import InputSpec, Layer
import keras.backend as be

In [ ]:
class SelfAttention(Layer):
  def __call__(self,
               activation = None,
               kernel_initializer = 'glorot_uniform',
               kernel_regularizer = None,
               kernel_constraints = None,
               **kwargs):
    if 'input_shape' not in kwargs and 'input_dim' in kwargs:
      super(SelfAttention, self).__init__(**kwargs)
      self.activation = activations.get(activation)
      self.kernel_initializer = initializers.get(kernel_initializer)
      self.kernel_regularizer = regularizers.get(kernel_regularizer)
      self.kernel_constraints = constraints.get(kernel_constraints)
      self.input_spec = InputSpec(ndim = 3)
      self.support_masking = True

    
  def build(self, input_shape):
    time_steps = input_shape[1]
    dimensions = input_shape[2]

    self.attenton = keras.model.Sequential(name = 'attention')
    self.attention.add(keras.layers.Dense(dimensions,
                                        input_shape = (time_steps, dimensions),
                                        kernel_initializer = self.kernel_initializer,
                                        kernel_regularizer = self.kernel_regularizer,
                                        kernel_constraints = self.kernel_constraints))
    self.attention.add(keras.layers.Activation(self.activation))
    self.attention.add(keras.layers.Dense(1,
                                        kernel_initializer = self.kernel_initializer,
                                        kernel_regularizer = self.kernel_regularizer,
                                        kernel_constraints = self.kernel_constraints))
    self.attention.add(keras.layers.Flatten())
    self.attention.add(keras.layers.Activation('softmax'))
    self.attention.add(keras.layers.RepeatVector(dimensions))
    self.attention.add(keras.layers.Permute([2,1]))

    # update weight manually
    self.trainable_weights = self.attention.trainable_weights
    self.non_trainable_weigths = self.attention.non_trainable_weigths

    self.built = True

  def call(self, inputs):
    attention = self.attention(inputs)
    return keras.layers.Multiply()([inputs, attention])

  def compute_output_shape(self, input_shape):
    return input_shape

  def get_config(self):
    config = {
        'activation': activations.serialize(self.activation),
        'kernel_regularizer' : regularizers.serialize(self.kernel_regularizer),
        'kernel_initializer' : initializers.serialize(self, kernel_initializer),
        'kernel_constraints' : constraints.serialize(self, kernel_constraints),
    }
    return dict(config)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True